In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'hatespeech-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5075104%2F8503427%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240531%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240531T064543Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D69e38690ab33e95c3c8f7d90998ed56c49bc77e15d3fbc3332096c4f8edf03f4898603800a7c084c109048544ca7f38058ccef0b237522f9869119512bcfe04e2bbb423341a551719d23752e6bba33186b4b18d33359702f05d91eb036542694d93ca2b6636fbceb4e4615eabf64f17567e33c6435afa04da9bc2f76a2e46bee4116b0502d5ce3a0b1ce9beac18e29f1ef75d9e846ef3553aea0bcc6d85dfb9bc260cfaacd7e25b32ee1ee6c29393b8c200e8cce4bacf0e79061c5a34795b3c45d3e4385c193cb0fe880f89c6e120ee6091f955cae74064e325ef6aeb1cd0333e8e1af1203cb08a34443bd7173505dc031e16ef9b83b2c55bae2b225c25827e1'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [1]:
#installing the dependencies
!pip install -q accelerate
!pip install -q transformers
!pip install -q peft
!pip install -q bitsandbytes
!pip install -q datasets
!pip install -q trl
!pip install huggingface_hub -q
!pip install datasets -q
!pip install nltk -q

In [2]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [30]:
nltk.data.path

['/root/nltk_data',
 '/usr/share/nltk_data',
 '/usr/local/share/nltk_data',
 '/usr/lib/nltk_data',
 '/usr/local/lib/nltk_data']

In [3]:
from datasets import load_dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig,
    DataCollatorWithPadding
)
from tqdm import tqdm
from trl import SFTTrainer
import torch
import time
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

2024-05-31 14:09:47.850097: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-31 14:09:47.850192: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-31 14:09:47.974999: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
hf_token = user_secrets.get_secret("HF_TOKEN")


from huggingface_hub import login
login(token = hf_token)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)
#torch.set_default_tensor_type(torch.cuda.FloatTensor)

In [6]:
data = load_dataset("tdavidson/hate_speech_offensive")
data = data['train']
data_df = data.to_pandas()

Generating train split:   0%|          | 0/24783 [00:00<?, ? examples/s]

In [6]:
#data_df = data_df[:5000]

In [7]:
# Load the stop words
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Lowercase the text
    text = text.lower()
    # Remove special characters and punctuation
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stop words and lemmatize the tokens
    tokens = [word for word in tokens if word not in stop_words]
    # Join the tokens back into a single string
    text = ' '.join(tokens)
    return text

# Apply the preprocessing function to the 'text' column
data_df['tweet'] = data_df['tweet'].apply(preprocess_text)

In [9]:
data_df

,count,hate_speech_count,offensive_language_count,neither_count,class,tweet
0,3,0,0,3,2,rt mayasolovely woman shouldnt complain cleani...
1,3,0,3,0,1,rt mleew17 boy dats coldtyga dwn bad cuffin da...
2,3,0,3,0,1,rt urkindofbrand dawg rt 80sbaby4life ever fuc...
3,3,0,2,1,1,rt cganderson vivabased look like tranny
4,6,0,6,0,1,rt shenikaroberts shit hear might true might f...
...,...,...,...,...,...,...
24778,3,0,2,1,1,yous muthafin lie 8220lifeasking 20pearls core...
24779,3,0,1,2,2,youve gone broke wrong heart baby drove rednec...
24780,3,0,3,0,1,young buck wan na eat dat nigguh like aint fuc...
24781,6,0,6,0,1,youu got wild bitches tellin lies


In [8]:
x = list(data_df['tweet'])
y = list(data_df['class'])

In [9]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )

In [10]:
model_name='microsoft/phi-2'
device_map = {"": 0}
model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                       num_labels = 3,
                                                      device_map=device_map,
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:468: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of PhiForSequenceClassification were not initialized from the model checkpoint at microsoft/phi-2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True,padding_side="left",add_eos_token=True,add_bos_token=True,use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
x_train, x_val, y_train, y_val = train_test_split(x,y, test_size = 0.3)
x_train_tokenized = tokenizer(x_train, padding = True, truncation = True, max_length = 512 )
x_val_tokenized = tokenizer(x_val, padding = True, truncation = True, max_length = 512 )

In [13]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(x_train_tokenized, y_train)
val_dataset = Dataset(x_val_tokenized, y_val)

In [14]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    clf_rep = classification_report(y_true=labels, y_pred=pred)

    return {"classification_report": clf_rep}

In [15]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

original_model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r= 1, #Rank
    lora_alpha=32,
    target_modules=[
        'q_proj',
        'k_proj',
        'v_proj',
        'dense'
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="SEQ_CLS",
)

# 1 - Enabling gradient checkpointing to reduce memory usage during fine-tuning
original_model.gradient_checkpointing_enable()

peft_model = get_peft_model(original_model, config)

In [17]:
peft_training_args = TrainingArguments(
    output_dir = "hate-phi",
    warmup_steps=1,
    num_train_epochs = 1,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    logging_steps=25,
    logging_dir="./logs",
    save_strategy="steps",
    save_steps=25,
    evaluation_strategy="steps",
    eval_steps=25,
    do_eval=True,
    gradient_checkpointing=True,
    report_to="none",
    overwrite_output_dir = 'True',
    group_by_length=True,
    
)

peft_model.config.use_cache = False

peft_trainer = Trainer(
    model=peft_model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    args=peft_training_args,
    data_collator= DataCollatorWithPadding(tokenizer),
    compute_metrics = compute_metrics
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [18]:
peft_trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Classification Report
25,0.612400,0.369616,precision recall f1-score support 0 0.44 0.03 0.05 422 1 0.90 0.95 0.93 5753 2 0.74 0.79 0.76 1260 accuracy 0.87 7435 macro avg 0.70 0.59 0.58 7435 weighted avg 0.85 0.87 0.85 7435
50,0.326800,0.290043,precision recall f1-score support 0 0.50 0.16 0.25 422 1 0.92 0.96 0.94 5753 2 0.82 0.85 0.84 1260 accuracy 0.89 7435 macro avg 0.75 0.66 0.67 7435 weighted avg 0.88 0.89 0.88 7435


<class 'transformers.trainer_utils.EvalPrediction'>


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


<class 'transformers.trainer_utils.EvalPrediction'>


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=68, training_loss=0.4219165900174309, metrics={'train_runtime': 18136.4921, 'train_samples_per_second': 0.957, 'train_steps_per_second': 0.004, 'total_flos': 5.399469532151808e+16, 'train_loss': 0.4219165900174309, 'epoch': 1.0})

In [19]:
peft_trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 0.2798629105091095,
 'eval_classification_report': '              precision    recall  f1-score   support\n\n           0       0.50      0.23      0.32       422\n           1       0.92      0.96      0.94      5753\n           2       0.84      0.85      0.85      1260\n\n    accuracy                           0.90      7435\n   macro avg       0.75      0.68      0.70      7435\nweighted avg       0.89      0.90      0.89      7435\n',
 'eval_runtime': 1210.0957,
 'eval_samples_per_second': 6.144,
 'eval_steps_per_second': 0.384,
 'epoch': 1.0}

In [ ]:
#Phi-2 metrics
'''
eval_classification_report': '              precision    recall  f1-score   support\n\n           0       0.50      0.23      0.32       422\n           1       0.92      0.96      0.94      5753\n           2       0.84      0.85      0.85      1260\n\n    accuracy                           0.90      7435\n   macro avg       0.75      0.68      0.70      7435\nweighted avg       0.89      0.90      0.89      7435\n',
'''

In [20]:
peft_trainer.push_to_hub("HatePhi-2")

adapter_model.safetensors:   0%|          | 0.00/2.69M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/datafreak/hate-phi/commit/a6ad6ce2bc2847d98d539573680891b47c05ab60', commit_message='HatePhi-2', commit_description='', oid='a6ad6ce2bc2847d98d539573680891b47c05ab60', pr_url=None, pr_revision=None, pr_num=None)

In [21]:
total_train_samples = 17000  # Replace with your dataset size
steps_per_epoch = total_train_samples // (peft_training_args.per_device_train_batch_size * peft_training_args.gradient_accumulation_steps)
target_training_steps = steps_per_epoch * peft_training_args.num_train_epochs  # 2 epochs

In [22]:
target_training_steps

264